In [1]:
!pip3 install bitsandbytes==0.41.3
!pip3 install peft==0.12.0
!pip3 install trl==0.8.6
!pip3 install datasets==2.19.2
!pip3 install transformers==4.43.3
!pip3 install tensorboard==2.17.0
!pip3 install accelerate==0.33.0

In [2]:
import os
import torch

from datasets import load_dataset
from transformers import (
    TrainingArguments,
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    logging,
    BitsAndBytesConfig
)
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [3]:
dataset_name = "Vezora/Tested-22k-Python-Alpaca"
model_id = "openai-community/gpt2"

# Load tokenizer

In [4]:
def load_tokenizer(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, use_fast=False)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    return tokenizer

# Quantization config for QLoRA

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16)

# Load model

In [6]:
def load_model(model_id, is_local=False):
    print("Loading: ", model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id, 
        #load_in_8bit=True,
        #quantization_config=bnb_config,
        device_map={"": 0},
        local_files_only = is_local
    )
    
    model.config.use_cache = False
    model.config.pretraining_tp = 1
    return model

# Generate Inference

In [7]:
def generate_inference(model, tokenizer):
    pipe = pipeline(task='text-generation', model=model, tokenizer=tokenizer, max_length=256)

    for prompt in prompts:
        result = pipe(prompt)
        print("******************************* PROMPT *******************************")
        print(prompt)
        print("******************************* Inference *******************************")
        print(result[0]['generated_text'])
        print("="*100)

# Test Inference Prompts

In [8]:
prompts = [
    """Can you design a program in Python that can predict the likelihood of a certain material becoming unstable under high pressure and temperature conditions? This can be calculated using the Gurney equation, which takes into account parameters such as the chemical composition and crystal structure of the material. Additionally, can you provide a real-world scenario where this program could be applied in the field of material science and engineering?""",
    """Write a Python function that returns the maximum value of the given set of integers: 1, 5, 23, and 9. The function should only use one line of code and must utilize the lambda function. Additionally, the function should be able to handle any set of integers provided as input, and should return an error message if any non-integer values are detected. The output should be displayed in a formatted table with the following columns: "Input Set", "Maximum Value". The table should also include a row for the given set of integers as well as a row for a randomly generated set of 10 integers between 1 and 100. Lastly, the function should be written in SQL and should be able to query a database table containing the input sets and their corresponding maximum values.""",
    """Create a function that takes two parameters, a number and a string, and returns an array with those two values in it. The string parameter should be checked if it is a valid string and if it contains only alphabetical characters. If the string is not valid (length less than 10 characters) or contains non-alphabetical characters, the function should return an empty array. Additionally, the number parameter should be restricted to a range between -1000 and 1000 (inclusive). The function should also check if the number is a prime number and return an empty array if it is not.""",
    """Given a string, convert it to upper case using only basic string manipulation operations. The string may contain punctuation marks, special characters, and spaces. String: This string should be in upper case!""",
    """Find the sum of the first 1000 prime numbers that are greater than 100.""",
    """Provide a function that finds the length of a string, but without using any built-in string length functions or methods, and without using any iteration or recursion.""",
    """Create a function that removes duplicates from an array and returns an array of the unique values. The function should only use O(1) additional space and have a time complexity of O(n), where n is the length of the input array. The input array can contain integers, floating-point numbers, strings, and nested arrays. The output array should be sorted in descending order. Additionally, the function should handle nested arrays correctly by recursively flattening them before removing duplicates.""",
    """Create a 3x4 NumPy array of random integers from 0 to 5, where each row should have at least one unique value.""",
    """Find the index of the element 'c' in the following list, but the list may contain duplicates and the element 'c' may appear multiple times.""",
    """Write a function for finding the minimum value in a given array, with a time complexity requirement of O(n log n), where n is the length of the array."""
]

# Pre Fine-tuning inference

In [9]:
model = load_model(model_id) 
tokenizer = load_tokenizer(model_id)
generate_inference(model, tokenizer)

Loading:  openai-community/gpt2


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


******************************* PROMPT *******************************
Can you design a program in Python that can predict the likelihood of a certain material becoming unstable under high pressure and temperature conditions? This can be calculated using the Gurney equation, which takes into account parameters such as the chemical composition and crystal structure of the material. Additionally, can you provide a real-world scenario where this program could be applied in the field of material science and engineering?
******************************* Inference *******************************
Can you design a program in Python that can predict the likelihood of a certain material becoming unstable under high pressure and temperature conditions? This can be calculated using the Gurney equation, which takes into account parameters such as the chemical composition and crystal structure of the material. Additionally, can you provide a real-world scenario where this program could be applied in 

******************************* PROMPT *******************************
Provide a function that finds the length of a string, but without using any built-in string length functions or methods, and without using any iteration or recursion.
******************************* Inference *******************************
Provide a function that finds the length of a string, but without using any built-in string length functions or methods, and without using any iteration or recursion. If the value is not zero, then append the value to the end of the string string and return the value. If the length is not longer than or equal to zero, then append the value to the end of the string which does not support a value of zero. For strings of length 0 to 255 inclusive, append the value to the end of the string in which the string has an index in which it may be found. An error on this string is a message that includes a zero-pointed value or the same line as an exit for the program.

In a special case, t

# PEFT Parameters

In [10]:
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT= 0.05
LORA_TARGET_MODULES = [
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "gate_proj",
    "up_proj",
    "down_proj",
    "lm_head",
]

BATCH_SIZE = 64
MICRO_BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 300

# LoRA parameters


In [11]:
peft_params = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM"
)

# Pre process fine-tune data record 

In [12]:
def preprocess_function(example):
    """
    Formatting function returning a list of samples (kind of necessary for SFT API).
    """
    text = f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}"
    return text

# Load Dataset

In [13]:
import datasets
from datasets import Dataset

dataset = load_dataset(dataset_name)

# Split Datasets into multiple equal parts

In [15]:
instruction = dataset["train"]["instruction"]
input = dataset["train"]["input"]
output = dataset["train"]["output"]

temp_dataset_1 = Dataset.from_dict({"instruction": instruction[:5500], "input": input[:5500], "output": output[:5500]})
dataset_1 = datasets.DatasetDict({"train": temp_dataset_1})

temp_dataset_2 = Dataset.from_dict({"instruction": instruction[5500:11000], "input": input[5500:11000], "output": output[5500:11000]})
dataset_2 = datasets.DatasetDict({"train": temp_dataset_2})

temp_dataset_3 = Dataset.from_dict({"instruction": instruction[11000:16500], "input": input[11000:16500], "output": output[11000:16500]})
dataset_3 = datasets.DatasetDict({"train": temp_dataset_3})

temp_dataset_4 = Dataset.from_dict({"instruction": instruction[16500:], "input": input[16500:], "output": output[16500:]})
dataset_4 = datasets.DatasetDict({"train": temp_dataset_4})


datasets = [dataset_1, dataset_2, dataset_3, dataset_4]

# test train split

In [16]:
def train_test_split(dataset):
    train_val = dataset["train"].train_test_split(test_size=500, shuffle=True, seed=42)
    train_data = train_val["train"]
    val_data = train_val["test"]

    return train_data, val_data

In [17]:
model = get_peft_model(model, peft_params)
model.print_trainable_parameters()

for param in model.parameters():
    if param.requires_grad:
        param.data = param.data.float()

trainable params: 816,400 || all params: 125,256,208 || trainable%: 0.6518


# Training Params

In [18]:
OUTPUT_DIR="/home/ec2-user/SageMaker/gpt-2/LoRA"
#OUTPUT_DIR="/kaggle/working/gpt-2"
training_args = TrainingArguments(
  output_dir=OUTPUT_DIR,
  num_train_epochs=3,
  per_device_train_batch_size=4,
  per_device_eval_batch_size=4,
  gradient_accumulation_steps=4,
  optim="paged_adamw_32bit",
  save_steps=200,
  logging_steps=200,
  learning_rate=0.0001,
  eval_strategy="steps",
  weight_decay=0.001,
  fp16=True,
  bf16=False,
  max_grad_norm=0.3,
  max_steps=-1,
  warmup_ratio=0.03,
  group_by_length=True,
  lr_scheduler_type="constant",
  report_to="tensorboard"
)

# Model trainer

In [19]:
def train(train_dataset, eval_dataset, model, tokenizer, output_path):
    
    trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        max_seq_length=context_length,
        tokenizer=tokenizer,
        args=training_args,
        formatting_func=preprocess_function,
        packing=True
    )
    
    print("Model output path: ", f"{output_path}/model/fine_tuned")
    
    history = trainer.train()
    model.save_pretrained(f"{output_path}/model/fine_tuned")
    #tokenizer.save_pretrained(f"{output_path}/tokenizer/fine_tuned")
    return model


In [20]:
context_length = 256

# finetune model with dataset 1

In [21]:
import datetime
import time

current_timestamp_ms = int(time.time() * 1000)
print("finetuning with dataset 1", current_timestamp_ms)

finetuning with dataset 1 1722937596958


In [22]:
BASE_DIR = "/home/ec2-user/SageMaker/gpt2/LoRA/dataset-"
#BASE_DIR = "/kaggle/working/gpt-2/dataset"
train_data, val_data = train_test_split(dataset_1)

model_id = "openai-community/gpt2"
index = 1
model = load_model(model_id)
tokenizer = load_tokenizer(model_id)
model = get_peft_model(model, peft_params)
model.print_trainable_parameters()
print("Fine-Tuning: ", model_id)
model = train(train_data, val_data, model, tokenizer, f"{BASE_DIR}{index}")

current_timestamp_ms = int(time.time() * 1000)
current_timestamp_ms

Loading:  openai-community/gpt2
trainable params: 816,400 || all params: 125,256,208 || trainable%: 0.6518
Fine-Tuning:  openai-community/gpt2


Generating train split: 0 examples [00:00, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1206 > 1024). Running this sequence through the model will result in indexing errors


Generating train split: 0 examples [00:00, ? examples/s]

Model output path:  /home/ec2-user/SageMaker/gpt2/LoRA/dataset-1/model/fine_tuned


Step,Training Loss,Validation Loss
200,2.409900,1.983344
400,2.204100,1.917339
600,2.150500,1.882504
800,2.107700,1.845392
1000,2.080400,1.812383
1200,2.042900,1.789211
1400,2.021000,1.772816
1600,2.007100,1.757129
1800,1.975600,1.745564
2000,1.971100,1.734569


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.

1722938042736

# inference after finetuning with dataset 1

In [23]:
model = load_model(f"{BASE_DIR}1/model/fine_tuned") 
#tokenizer = load_tokenizer(f"{BASE_DIR}1/tokenizer/fine_tuned")
generate_inference(model, tokenizer)

Loading:  /home/ec2-user/SageMaker/gpt2/LoRA/dataset-1/model/fine_tuned


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


******************************* PROMPT *******************************
Can you design a program in Python that can predict the likelihood of a certain material becoming unstable under high pressure and temperature conditions? This can be calculated using the Gurney equation, which takes into account parameters such as the chemical composition and crystal structure of the material. Additionally, can you provide a real-world scenario where this program could be applied in the field of material science and engineering?
******************************* Inference *******************************
Can you design a program in Python that can predict the likelihood of a certain material becoming unstable under high pressure and temperature conditions? This can be calculated using the Gurney equation, which takes into account parameters such as the chemical composition and crystal structure of the material. Additionally, can you provide a real-world scenario where this program could be applied in 

******************************* PROMPT *******************************
Create a function that removes duplicates from an array and returns an array of the unique values. The function should only use O(1) additional space and have a time complexity of O(n), where n is the length of the input array. The input array can contain integers, floating-point numbers, strings, and nested arrays. The output array should be sorted in descending order. Additionally, the function should handle nested arrays correctly by recursively flattening them before removing duplicates.
******************************* Inference *******************************
Create a function that removes duplicates from an array and returns an array of the unique values. The function should only use O(1) additional space and have a time complexity of O(n), where n is the length of the input array. The input array can contain integers, floating-point numbers, strings, and nested arrays. The output array should be sorted in des

# Finetune with model with dataset 2

In [24]:
current_timestamp_ms = int(time.time() * 1000)
print("dataset 2 finetuning", current_timestamp_ms)

dataset 2 finetuning 1722938086160


In [25]:
train_data, val_data = train_test_split(dataset_2)

model_id = f"{BASE_DIR}1/model/fine_tuned"
index = 2
model = load_model(model_id, True)
model = get_peft_model(model, peft_params)
model.print_trainable_parameters()
print("Fine-Tuning: ", model_id)
model = train(train_data, val_data, model, tokenizer, f"{BASE_DIR}{index}")

current_timestamp_ms = int(time.time() * 1000)
current_timestamp_ms

Loading:  /home/ec2-user/SageMaker/gpt2/LoRA/dataset-1/model/fine_tuned
trainable params: 816,400 || all params: 125,256,208 || trainable%: 0.6518
Fine-Tuning:  /home/ec2-user/SageMaker/gpt2/LoRA/dataset-1/model/fine_tuned


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Model output path:  /home/ec2-user/SageMaker/gpt2/LoRA/dataset-2/model/fine_tuned


Step,Training Loss,Validation Loss
200,2.439500,2.000025
400,2.217400,1.917570
600,2.163100,1.875599
800,2.118100,1.833469
1000,2.063000,1.800728
1200,2.035500,1.772283
1400,2.003000,1.751814
1600,1.983100,1.736442
1800,1.977300,1.720999


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.

1722938375140

# inference after finetuning with dataset 2

In [26]:
model = load_model(f"{BASE_DIR}2/model/fine_tuned") 
generate_inference(model, tokenizer)

Loading:  /home/ec2-user/SageMaker/gpt2/LoRA/dataset-2/model/fine_tuned
******************************* PROMPT *******************************
Can you design a program in Python that can predict the likelihood of a certain material becoming unstable under high pressure and temperature conditions? This can be calculated using the Gurney equation, which takes into account parameters such as the chemical composition and crystal structure of the material. Additionally, can you provide a real-world scenario where this program could be applied in the field of material science and engineering?
******************************* Inference *******************************
Can you design a program in Python that can predict the likelihood of a certain material becoming unstable under high pressure and temperature conditions? This can be calculated using the Gurney equation, which takes into account parameters such as the chemical composition and crystal structure of the material. Additionally, can y

******************************* PROMPT *******************************
Provide a function that finds the length of a string, but without using any built-in string length functions or methods, and without using any iteration or recursion.
******************************* Inference *******************************
Provide a function that finds the length of a string, but without using any built-in string length functions or methods, and without using any iteration or recursion. Additionally, implement a recursion that takes the shortest representation to find the length.
Here's an implementation of recursion that takes a function where we loop through each item in a string.
******************************* PROMPT *******************************
Create a function that removes duplicates from an array and returns an array of the unique values. The function should only use O(1) additional space and have a time complexity of O(n), where n is the length of the input array. The input array can co

# Finetune with model with dataset 3

In [27]:
current_timestamp_ms = int(time.time() * 1000)
print("dataset 3 finetuning", current_timestamp_ms)

dataset 3 finetuning 1722938536078


In [30]:
train_data, val_data = train_test_split(dataset_3)

model_id = f"{BASE_DIR}2/model/fine_tuned"
index = 3
model = load_model(model_id, True)
model = get_peft_model(model, peft_params)
model.print_trainable_parameters()
print("Fine-Tuning: ", model_id)
model = train(train_data, val_data, model, tokenizer, f"{BASE_DIR}{index}")

current_timestamp_ms = int(time.time() * 1000)
current_timestamp_ms

Loading:  /home/ec2-user/SageMaker/gpt2/LoRA/dataset-2/model/fine_tuned
trainable params: 816,400 || all params: 125,256,208 || trainable%: 0.6518
Fine-Tuning:  /home/ec2-user/SageMaker/gpt2/LoRA/dataset-2/model/fine_tuned
Model output path:  /home/ec2-user/SageMaker/gpt2/LoRA/dataset-3/model/fine_tuned


Step,Training Loss,Validation Loss
200,2.559500,2.128304
400,2.352600,2.041639
600,2.276200,1.988267
800,2.224600,1.953032
1000,2.200700,1.925541
1200,2.153800,1.901576
1400,2.140700,1.882540
1600,2.126700,1.871080
1800,2.099300,1.860218
2000,2.099100,1.848099


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.

1722939326516

# inference after finetuning with dataset 3

In [29]:
model = load_model(f"{BASE_DIR}3/model/fine_tuned") 
generate_inference(model, tokenizer)

Loading:  /home/ec2-user/SageMaker/gpt2/LoRA/dataset-3/model/fine_tuned
******************************* PROMPT *******************************
Can you design a program in Python that can predict the likelihood of a certain material becoming unstable under high pressure and temperature conditions? This can be calculated using the Gurney equation, which takes into account parameters such as the chemical composition and crystal structure of the material. Additionally, can you provide a real-world scenario where this program could be applied in the field of material science and engineering?
******************************* Inference *******************************
Can you design a program in Python that can predict the likelihood of a certain material becoming unstable under high pressure and temperature conditions? This can be calculated using the Gurney equation, which takes into account parameters such as the chemical composition and crystal structure of the material. Additionally, can y

******************************* PROMPT *******************************
Find the sum of the first 1000 prime numbers that are greater than 100.
******************************* Inference *******************************
Find the sum of the first 1000 prime numbers that are greater than 100. It cannot be greater than the sum of those that are equal to 10. Find each 1-in-1 as a subsequence of all the numbers within the sequence of prime numbers that are in the sequence of prime numbers. If all prime numbers have the same length (10), it is equivalent to find if the prime number is in the prime number range (0, 10, 30, 60, 90, 120, 180, 240, 500, 1000, 1000:0, 2000), otherwise find prime number within the sequence.
To find all integer numbers, calculate the value of the sequence of numbers in the range 10 to 10, inclusive, in the sequence 10 to 10, inclusive, in the sequence 10 to 7 (excluding the remainder of the sequence) and compare the product of the numbers with the sum of the prime num

# Finetune with model with dataset 4

In [31]:
current_timestamp_ms = int(time.time() * 1000)
print("dataset 4 finetuning", current_timestamp_ms)

dataset 4 finetuning 1722939340914


In [32]:
train_data, val_data = train_test_split(dataset_4)

model_id = f"{BASE_DIR}3/model/fine_tuned"
index = 4
model = load_model(model_id, True)
model = get_peft_model(model, peft_params)
model.print_trainable_parameters()
print("Fine-Tuning: ", model_id)
model = train(train_data, val_data, model, tokenizer, f"{BASE_DIR}{index}")

current_timestamp_ms = int(time.time() * 1000)
current_timestamp_ms

Loading:  /home/ec2-user/SageMaker/gpt2/LoRA/dataset-3/model/fine_tuned
trainable params: 816,400 || all params: 125,256,208 || trainable%: 0.6518
Fine-Tuning:  /home/ec2-user/SageMaker/gpt2/LoRA/dataset-3/model/fine_tuned


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Model output path:  /home/ec2-user/SageMaker/gpt2/LoRA/dataset-4/model/fine_tuned


Step,Training Loss,Validation Loss
200,3.007100,2.495898
400,2.648600,2.254859
600,2.545000,2.183419
800,2.484500,2.137817
1000,2.435700,2.106049
1200,2.400600,2.079571
1400,2.370100,2.059118
1600,2.354300,2.040565
1800,2.346000,2.024897
2000,2.320100,2.013632


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.

1722939806146

# inference after finetuning with dataset 4

In [33]:
model = load_model(f"{BASE_DIR}4/model/fine_tuned") 
generate_inference(model, tokenizer)

Loading:  /home/ec2-user/SageMaker/gpt2/LoRA/dataset-4/model/fine_tuned
******************************* PROMPT *******************************
Can you design a program in Python that can predict the likelihood of a certain material becoming unstable under high pressure and temperature conditions? This can be calculated using the Gurney equation, which takes into account parameters such as the chemical composition and crystal structure of the material. Additionally, can you provide a real-world scenario where this program could be applied in the field of material science and engineering?
******************************* Inference *******************************
Can you design a program in Python that can predict the likelihood of a certain material becoming unstable under high pressure and temperature conditions? This can be calculated using the Gurney equation, which takes into account parameters such as the chemical composition and crystal structure of the material. Additionally, can y

******************************* PROMPT *******************************
Provide a function that finds the length of a string, but without using any built-in string length functions or methods, and without using any iteration or recursion.
******************************* Inference *******************************
Provide a function that finds the length of a string, but without using any built-in string length functions or methods, and without using any iteration or recursion.


I've previously written a Python function where the first argument is just, or the "hello" parameter is a dictionary object with length.




















Finally, to create a function that prints out the length and output of each function it would use, we need a function that prints the first three elements of an array in a Python variable.



























 It all sounds straightforward, but Python's functions are not quite at the cutting edge of Python's performance. They struggle to handle large nu